In [55]:
import bz2
import functools
import pickle

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import wikipedia2vec

In [2]:
with bz2.open("../enwiki_20180420_100d.pkl.bz2", "rb") as f:
    wiki2vec = wikipedia2vec.Wikipedia2Vec.load(f)

/home/ocifka/miniconda3/envs/wiki2vec/lib/python3.8/contextlib.py:113: UserWarning: "<bz2.BZ2File object at 0x7f26406e2520>" is not a raw file, mmap_mode "c" flag will be ignored.
  return next(self.gen)


In [4]:
individuals = pd.read_parquet("individuals.parquet")

In [18]:
# Manually defined cases that are not resolved automatically
TAXON_MAP = {
    "Rissa": "Kittiwake"
}

In [33]:
taxon2emb = {}

for taxon, genus in individuals[["taxon_canonical_name", "taxon_genus"]].drop_duplicates().itertuples(index=False):
    print(taxon, end=" ")
    entity = None
    if taxon in TAXON_MAP:
        entity = wiki2vec.get_entity(TAXON_MAP[taxon])
        print("->", TAXON_MAP[taxon], end=" ")
    else:
        entity = wiki2vec.get_entity(taxon)
        if entity is None and genus is not None:
            entity = wiki2vec.get_entity(genus)
            print("->", genus, end=" ")
    if entity is None:
        print("-> FAIL")
    else:
        print("->", entity)
        taxon2emb[taxon] = wiki2vec.get_vector(entity)

Uria lomvia -> <Entity Thick-billed murre>
Columba livia -> <Entity Rock dove>
Larus marinus -> <Entity Great black-backed gull>
Catharus minimus -> <Entity Grey-cheeked thrush>
 -> FAIL
Ciconia ciconia -> <Entity White stork>
Pandion haliaetus -> <Entity Osprey>
Melanitta nigra -> <Entity Common scoter>
Homo sapiens -> <Entity Homo sapiens>
Balaenoptera musculus -> <Entity Blue whale>
Balaenoptera physalus -> <Entity Fin whale>
Cygnus columbianus -> <Entity Tundra swan>
Felis catus -> <Entity Cat>
Anser albifrons -> <Entity Greater white-fronted goose>
Milvus migrans -> <Entity Black kite>
Agkistrodon contortrix -> <Entity Agkistrodon contortrix>
Larus argentatus -> <Entity European herring gull>
Buteo jamaicensis -> <Entity Red-tailed hawk>
Pteropus poliocephalus -> <Entity Grey-headed flying fox>
Numenius arquata -> <Entity Eurasian curlew>
Tockus deckeni -> <Entity Von der Decken's hornbill>
Lynx rufus -> <Entity Bobcat>
Larus -> <Entity Larus>
Larus smithsonianus -> <Entity Americ

In [40]:
nan_emb = next(iter(taxon2emb.values())) * np.nan

In [59]:
individual2emb = {
    ind: taxon2emb[taxon] if taxon in taxon2emb else nan_emb
    for ind, taxon in individuals[["id", "taxon_canonical_name"]].drop_duplicates().itertuples(index=False)
}

with open("individual2taxon_emb.pickle", "wb") as f:
    pickle.dump(individual2emb, f)